# Проект с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель и добьемся на ней значения метрики качества F1 не меньше 0.75. 

## Загрузка и предобработка данных

In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertConfig, BertModel
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")

D:\anaconda\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\casa\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
comments = pd.read_csv('D:\\data\\toxic_comments.csv', index_col=[0])
comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [19]:
comments = comments.sample(1000).reset_index(drop=True) 

Токенизируем текст, чтобы разбить его на эмбединги

In [7]:
tokenized = comments['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [8]:
configuration = BertConfig()

model = BertModel.from_pretrained("unitary/toxic-bert")

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
features = np.concatenate(embeddings)
target = comments['toxic']

print(features.shape, target.shape)

(1000, 768) (1000,)


Теперь разделим выборку на обучающую и тестовую и приступим к обучению моделей.

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    features,target,test_size=0.2,shuffle=True,random_state=42)

### RandomForrestClassifier

In [12]:
param_grid_ = { # словарь параетров
    'max_depth':[5,7,9,11,13,15,17,19,20],
    'min_samples_leaf':[2,4,6,8],    
    'n_estimators': [10, 20, 30, 40, 50],
    'max_features': ['log2'],    
    'criterion' : ['gini']
}

In [14]:
%%time
rfc = RandomForestClassifier(random_state=42)
cv_rfc = GridSearchCV(estimator=rfc,
                       param_grid=param_grid_,
                       cv=5,
                       scoring='f1'
)
cv_rfc.fit(features_train, target_train)
#preds_rfc = grid_CV_rfc.best_estimator_.predict(X_train)
print(cv_rfc.best_score_)

0.966993490532912
Wall time: 1min 1s


### DecisionTreeClassifier

In [15]:
%%time
dtf = DecisionTreeClassifier(random_state=42)#Исследуем модель дерева решений
param_grid_dtf = { # словарь параметров
    'max_depth':[5,7,9,11,13,15,17,19,20],
    'min_samples_leaf':[1, 2, 3, 4, 6, 8],
    'min_samples_split' : [2, 3, 4, 5]
}

#воспользуемся методом GridSearchCV для подбора параметров
cv_dtf = GridSearchCV(estimator=dtf,
                       param_grid=param_grid_dtf,
                       cv=5,
                       scoring='f1'
)
cv_dtf.fit(features_train, target_train)


print(cv_dtf.best_score_)

0.9578521459633201
Wall time: 1min 24s


### LogisticRegression

In [17]:
param = { 'C': range(1, 11, 2), 'class_weight': [None, 'balanced'] }
model_lr = LogisticRegression()

# инициализируем GridSearchCV
cv_lr = GridSearchCV(estimator = model_lr, 
                           param_grid = param, 
                           cv = 3,
                           n_jobs = -1, 
                           verbose = 0, 
                           scoring = 'f1',
                          )
cv_lr.fit(features_train, target_train)    
print(cv_lr.best_score_)

0.9626583104843975


## Проверим лучшую модель на тестовой выборке

In [18]:
f1_score(target_test, cv_rfc.predict(features_test))

0.9090909090909091

# Вывод

Для предобработки текста мы взяли предобученную модель BERT на токсичных комментариях.
Воспользовавшись ей, разбили текст на эмбеддинги, применили разные модели классификации, добились требуемого значения метрики
`f1` - 0.9091